In [70]:
from hossam import *

from pandas import DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression

from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)
from scipy.stats import t, f
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

import statsmodels.api as sm


In [71]:
origin = load_data("fish_processed")

yname = "무게"
x_train = origin.drop(columns=[yname])
y_train = origin[yname]

display(origin.head())

농어의 길이,높이,두께,무게를 조사한 데이터의 전처리 버전


,길이,높이,두께,무게
0,-2.180225,-2.016507,-1.896175,1.931521
1,-1.587434,-1.518703,-1.560774,3.496508
2,-1.442032,-1.417039,-1.316328,3.713572
3,-1.307815,-1.147103,-1.202633,3.960813
4,-1.173599,-1.147103,-1.026405,4.262680


In [72]:
estimator = LinearRegression(n_jobs=-1)
estimator.fit(x_train, y_train)
estimator

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",-1
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [73]:
yname= "무게"
xnames= list(origin.drop(yname,axis=1 ).columns)
yname, xnames

('무게', ['길이', '높이', '두께'])

### 잔차구하기

In [74]:
y_train_pred = estimator.predict(x_train)

#잔차 계산
resid = y_train - y_train_pred
resid[:5]

0   -1.307725
1   -0.325159
2   -0.263963
3   -0.172176
4    0.002476
Name: 무게, dtype: float64

### durbin_watson 구하기

In [75]:
dw = durbin_watson(resid)
dw

np.float64(0.5639828667071892)

### 설명력

In [76]:
r2 = r2_score(y_train, y_train_pred)
r = np.sqrt(r2)
adj_r2 = 1-(1-r2)*(len(y_train) -1)/(len(y_train)-len(xnames)-1)
r2, adj_r2

(0.9485313491284232, 0.9455620038858322)

### F-통계량과 p-value

In [77]:
#표본수
rowcount = len(x_train)

#독립변수의 수
featurecount = len(xnames)

#F-statistic
f_statistic = (r2/ featurecount)/((1-r2)/ (rowcount - featurecount -1))

#Prob(F-statistic)
p=1-f.cdf(f_statistic, featurecount, rowcount - featurecount -1)

print(f"F-statistic: {f_statistic}")
print(f"p-value: {p}")

F-statistic: 319.44124769431
p-value: 1.1102230246251565e-16


In [78]:
rdf =DataFrame(
    {
        "R": [r],
        "R**2" : [r2],
        "Adj R**2" : [adj_r2],
        "F": [f_statistic],
        "p-value":[p],
        "Durbin-Watson": [dw],
    }
)
rdf

,R,R**2,Adj R**2,F,p-value,Durbin-Watson
0,0.973926,0.948531,0.945562,319.441248,1.110223e-16,0.563983


In [79]:
tpl = "R(%.3f), R**2(%.3f), ADJ R**2(%.3f), F(%.3f), P-value(%.3f), Durbin-Watson(%.3f)"
tpl % (r, r2, adj_r2, f_statistic, p, dw)

'R(0.974), R**2(0.949), ADJ R**2(0.946), F(319.441), P-value(0.000), Durbin-Watson(0.564)'

In [80]:
tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s, %s)= %0.3f, p%s 0.05)."

tpl % (
    yname,
    ",".join(xnames),
    "유의하다"if p <=0.05 else "유의하지 않다",
    len(x_train.columns),
    len(x_train.index) - len(x_train.columns)-1,
    f_statistic,
    "<=" if p<=0.05 else ">",
    )

'무게에 대하여 길이,높이,두께로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(3, 52)= 319.441, p<= 0.05).'

# 독립변수 통계량

In [81]:
params = np.append(estimator.intercept_, estimator.coef_)
params

array([5.46864075, 0.82149299, 0.12690658, 0.0962184 ])

In [82]:
design_x = x_train.copy()
design_x = sm.add_constant(design_x)
design_x.head()

,const,길이,높이,두께
0,1.0,-2.180225,-2.016507,-1.896175
1,1.0,-1.587434,-1.518703,-1.560774
2,1.0,-1.442032,-1.417039,-1.316328
3,1.0,-1.307815,-1.147103,-1.202633
4,1.0,-1.173599,-1.147103,-1.026405


### 독립변수의 행렬곱


In [83]:
dot = np.dot(design_x.T, design_x)
dot

array([[ 5.60000000e+01, -1.04360964e-14,  8.21565038e-15,
        -8.88178420e-16],
       [-1.04360964e-14,  5.60000000e+01,  5.51947045e+01,
         5.45761176e+01],
       [ 8.21565038e-15,  5.51947045e+01,  5.60000000e+01,
         5.50391971e+01],
       [-8.88178420e-16,  5.45761176e+01,  5.50391971e+01,
         5.60000000e+01]])

### 행렬곱의 역행렬

In [84]:
inv = np.linalg.inv(dot)
inv

array([[ 1.78571429e-02,  1.96685433e-16, -2.45724118e-16,
         5.01069847e-17],
       [ 1.96685433e-16,  6.48339391e-01, -5.29174626e-01,
        -1.11758935e-01],
       [-2.45724118e-16, -5.29174626e-01,  9.56813327e-01,
        -4.24677510e-01],
       [ 5.01069847e-17, -1.11758935e-01, -4.24677510e-01,
         5.44165678e-01]])

### 역행렬의 대각선 반환

In [85]:
dia = inv.diagonal()
dia

array([0.01785714, 0.64833939, 0.95681333, 0.54416568])

### 평균 제곱오차 구하기

In [86]:
predictions = estimator.predict(x_train)
n= design_x.shape[0]
p = design_x.shape[1]
MSE = ((y_train - predictions) **2).sum()/(n-p)
MSE

np.float64(0.0633316530564705)

### 표준 오차

In [87]:
se_b = np.sqrt(MSE *dia)
se_b

array([0.03362919, 0.20263367, 0.2461637 , 0.18564189])

### t-value 구하기

In [88]:
ts_b = params/se_b
ts_b

array([162.61589576,   4.05407935,   0.51553733,   0.5183011 ])

### p-value 구하기

In [89]:
n = design_x.shape[0]
p = design_x.shape[1]
p_values = [2*(1-t.cdf(np.abs(i), n-p)) for i in ts_b]
p_values

[np.float64(0.0),
 np.float64(0.0001689567475446907),
 np.float64(0.6083625770474206),
 np.float64(0.6064467034237375)]

### vif 구하기

In [90]:
vif = []

for i,v in enumerate(xnames):
    j = list(x_train.columns).index(v)
    vif.append(variance_inflation_factor(x_train, j))

print(vif)

[np.float64(36.30700589561463), np.float64(53.5815462967705), np.float64(30.473277956534112)]


## 표준화 계수(베타)구하기

In [91]:
scaler = StandardScaler()
std = scaler.fit_transform(origin)
std_df = DataFrame(std, columns=origin.columns)
std_df.head()

,길이,높이,두께,무게
0,-2.180225,-2.016507,-1.896175,-3.309048
1,-1.587434,-1.518703,-1.560774,-1.844971
2,-1.442032,-1.417039,-1.316328,-1.641903
3,-1.307815,-1.147103,-1.202633,-1.410604
4,-1.173599,-1.147103,-1.026405,-1.128201


In [92]:
scaler = StandardScaler()
std =scaler.fit_transform(origin)
std_df = DataFrame(std, columns=origin.columns)
std_x = std_df.drop(columns=[yname])
std_y = std_df[yname]
s_estimator = LinearRegression(n_jobs=-1)
s_estimator.fit(std_x, std_y)
beta = s_estimator.coef_
beta

array([0.76852356, 0.11872371, 0.09001429])

### 결과표 구성하기

In [93]:
result_df= DataFrame(
    {
        "종속변수": [yname]*len(xnames),
        "독립변수":xnames, 
        "B(비표준화 계수)":np.round(params[1:], 4),
        "표준오차": np.round(se_b[1:],4),
        "표준화계수":np.round(beta, 4),
        "t":np.round(ts_b[1:], 4),
        "유의확률":np.round(p_values[1:], 4),
        "VIF":np.round(vif, 4),
    }
)

#유의확률에 따라 t별표 추가
result_df["t"]= result_df["t"].astype("str") + result_df["유의확률"].apply(
    lambda p: "***" if p<0.001 else "**" if p<0.01 else "*" if p<0.05 else ""
)
result_df

,종속변수,독립변수,B(비표준화 계수),표준오차,표준화계수,t,유의확률,VIF
0,무게,길이,0.8215,0.2026,0.7685,4.0541***,0.0002,36.3070
1,무게,높이,0.1269,0.2462,0.1187,0.5155,0.6084,53.5815
2,무게,두께,0.0962,0.1856,0.0900,0.5183,0.6064,30.4733


In [94]:
varstr = []

for n in xnames:
    item = result_df[result_df["독립변수"] == n]
    coef = float(item["B(비표준화 계수)"].values[0])
    pvalue = float(item["유의확률"].values[0])

    s="%s가 1증가하면 %s(이)가 %0.3f만큼 변하는 것으로 나타남(p %s 0.05, %s)"
    k= s%(
        n,
        yname,
        coef,
        "<=" if pvalue <=0.05 else ">",
        ("유의함" if pvalue<=0.05 else "유의하지 않음"),
    )
    varstr.append(k)

varstr

['길이가 1증가하면 무게(이)가 0.822만큼 변하는 것으로 나타남(p <= 0.05, 유의함)',
 '높이가 1증가하면 무게(이)가 0.127만큼 변하는 것으로 나타남(p > 0.05, 유의하지 않음)',
 '두께가 1증가하면 무게(이)가 0.096만큼 변하는 것으로 나타남(p > 0.05, 유의하지 않음)']